In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
import pandas as pd

In [3]:
url='https://www.meloman.kz/catalogsearch/result/index/q/*/?amp%3Bproduct_list_order=product_rate&%3Bq=%2A&%3Bsearch_cat=&cat=1172&product_list_order=product_rate'


In [20]:
geckodriver_path = r'/home/sdu/Desktop/lessons/AP/geckodriver'
service = Service(geckodriver_path)
driver = webdriver.Firefox(service=service)

In [21]:
names = []
prices = []
genres=[]
years=[]
publishers=[]
ages=[]
languages=[]
ratings=[]
counts=[]

In [22]:
book_number=0

In [25]:
# i == page number
# n == number of pages
# there's 100 books in each page

n=10

# loading page
for i in range(1,n+1):
    
    driver.get(url+'&p='+str(i))

    soup = BeautifulSoup(driver.page_source)
    
    for book_block in soup.findAll('div', attrs={'class':'product details product-item-details'}):
        book_number+=1

        name = book_block.find('a',attrs={'class':'product-item-link'})
        price = book_block.find('span', attrs={'class': 'price'}).find('span')

#         load book information separately one by one
        link = name.get('href')

        driver.get(link)
        
        book_content=BeautifulSoup(driver.page_source)
        if (price==None or price == ''):
            price = book_content.find('span', attrs={'class': 'price'})

        rating= book_content.find('div',attrs={'class':'product-reviews-summary'})

        rating_percent=rating.find('div',attrs={'class':'rating-result'})
        rating_count=rating.find('span',attrs={'class':'counter'})

        info = book_content.find('table',attrs={'class':'additional-attributes'})

        try:
            publisher = info.find('td',attrs={'data-th':'Издательство'}).find('a')
        except:
            publisher = info.find('td',attrs={'data-th':'Издательство'})

        genre=info.find('td',attrs={'data-th':'Жанр'})
        age=info.find('td',attrs={'data-th':'Возраст'})
        year=info.find('td',attrs={'data-th':'Год издания'})
        language=info.find('td',attrs={'data-th':'Язык'})
            
        names.append(name.text if name else '')
        prices.append(int(''.join(price.text.split('\xa0')[:-1])) if price else '')
        ratings.append(rating_percent.get('title') if rating_percent else '')
        counts.append(int(rating_count.text) if rating_count else '')
        publishers.append(publisher.text if publisher else '')
        genres.append(genre.text if genre else '')
        ages.append(age.text if age else '')
        years.append(int(year.text) if year else '')
        languages.append(language.text if language else '')

        print('Done book: '+str(book_number))
    print('Done page:'+str(i))

1-go,0-exit0


MaxRetryError: HTTPConnectionPool(host='localhost', port=55673): Max retries exceeded with url: /session/937c306c-dca2-4909-9b99-3ec2e7f6bfb5/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0bd33d4160>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [8]:
data = {
    'name': names,
    'price': prices,
    'rating':ratings,
    'count':counts,
    'genre':genres,
    'publisher':publishers,
    'age':ages,
    'year':years,
    'language':languages
}

In [15]:
df = pd.DataFrame(data)
df.to_csv('data.csv', index=False)